# Recursive serving

recursive_serve_flow is a util function that recursively serves all local subflows of the flow you intend to serve. This is helpful when working with deeply nested flows, as it means you don't have to manually serve individual subflows. recursive_serve_flow will fetch the default config of the flow you are trying to serve, iterate over its subflows and recursively call itself on each subflow that specifies a local `user_id` (or doesn't have user_id field). 

recursive_serve_flow expects that a subflow config will contain fields corresponding to arguments needed to call recursive_serve_flow on that subflow. This means that along with the standard `user_id` and `flow_endpoint` fields, the subflow configs should contain the `flow_class_name` field so that recursive_serve knows what to serve at that endpoint. Recursive serve will also pick up additional serve arguments from the subflow configs (`singleton`, `dispatch_point`, `parallel_dispatch`), or give them default values if missing.

In [1]:
%load_ext autoreload
%autoreload 2
import os, json
from colink import CoLink
from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread

In [2]:
cl = colink_utils.start_colink_server()

### Start a few default workers

In [3]:
run_dispatch_worker_thread(cl)
run_dispatch_worker_thread(cl)

[2024-03-23 04:10:53,809][aiflows.workers.dispatch_worker:235][INFO] - Dispatch worker started in attached thread.
[2024-03-23 04:10:53,811][aiflows.workers.dispatch_worker:236][INFO] - dispatch_point: coflows_dispatch
[2024-03-23 04:10:53,826][aiflows.workers.dispatch_worker:235][INFO] - Dispatch worker started in attached thread.
[2024-03-23 04:10:53,827][aiflows.workers.dispatch_worker:236][INFO] - dispatch_point: coflows_dispatch
[2024-03-23 04:10:54,573][aiflows.workers.dispatch_worker:119][INFO] - 
~~~ Dispatch task ~~~
[2024-03-23 04:10:54,576][aiflows.workers.dispatch_worker:161][INFO] - flow_endpoint: reverse_number_sequential
[2024-03-23 04:10:54,577][aiflows.workers.dispatch_worker:162][INFO] - flow_id: 2ea02f80-7f58-403b-bdc8-af2fc02b2364
[2024-03-23 04:10:54,578][aiflows.workers.dispatch_worker:163][INFO] - owner_id: local
[2024-03-23 04:10:54,579][aiflows.workers.dispatch_worker:164][INFO] - message_paths: ['push_tasks:2b1528b8-4e55-404f-a2e6-734e8fc0fb37:msg']
[2024-03-2

### Observe default config of the Flow we will serve

Note the `flow_class_name` and `singleton` fields of the first_reverse_flow subflow. 

In [8]:
cfg = read_yaml_file("ReverseNumberFlowModule/ReverseNumberSequentialFlow.yaml")
print(json.dumps(cfg, indent=4))

{
    "name": "ReverseNumberTwice",
    "description": "A sequential flow that reverses a number twice.",
    "_target_": "ReverseNumberFlowModule.ReverseNumberSequentialFlow.instantiate_from_default_config",
    "input_interface": [
        "number"
    ],
    "output_interface": [
        "output_number"
    ],
    "subflows_config": {
        "first_reverse_flow": {
            "user_id": "local",
            "flow_endpoint": "reverse_number_atomic",
            "flow_class_name": "ReverseNumberFlowModule.ReverseNumberAtomicFlow",
            "singleton": true,
            "name": "ReverseNumberFirst",
            "description": "A flow that takes in a number and reverses it."
        },
        "second_reverse_flow": {
            "user_id": "local",
            "flow_endpoint": "reverse_number_atomic",
            "flow_class_name": "ReverseNumberFlowModule.ReverseNumberAtomicFlow",
            "name": "ReverseNumberSecond",
            "description": "A flow that takes in a numbe

### Call recursive serve

In [4]:
serve_utils.recursive_serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential"
)
# ReverseNumberAtomicFlow gets automatically served

[2024-03-23 04:10:54,032][aiflows.utils.serve_utils:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
[2024-03-23 04:10:54,033][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-23 04:10:54,034][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-23 04:10:54,034][aiflows.utils.serve_utils:119][INFO] - singleton: True

[2024-03-23 04:10:54,044][aiflows.utils.serve_utils:716][INFO] - Subflow second_reverse_flow already served.
[2024-03-23 04:10:54,144][aiflows.utils.serve_utils:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
[2024-03-23 04:10:54,146][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-23 04:10:54,148][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-23 04:10:54,148][aiflows.utils.serve_utils:119][INFO] - singleton: False



True

### Get instance

In [5]:
flow = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="reverse_number_sequential",
    user_id="local",
)

[2024-03-23 04:10:54,339][aiflows.utils.serve_utils:336][INFO] - Mounted e657de44-7a8d-455a-8dc5-50c50bcb8181 at flows:reverse_number_atomic:mounts:local:e657de44-7a8d-455a-8dc5-50c50bcb8181
[2024-03-23 04:10:54,398][aiflows.utils.serve_utils:543][INFO] - Fetched singleton e657de44-7a8d-455a-8dc5-50c50bcb8181
[2024-03-23 04:10:54,460][aiflows.utils.serve_utils:336][INFO] - Mounted 2ea02f80-7f58-403b-bdc8-af2fc02b2364 at flows:reverse_number_sequential:mounts:local:2ea02f80-7f58-403b-bdc8-af2fc02b2364


In [6]:
input_data = {"id": 0, "number": 1234}
    
input_message = flow.package_input_message(input_data)
reply_data = flow.get_reply_future(input_message).get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

Data sent:
 {'id': 0, 'number': 1234} 

REPLY:
 {'output_number': 1234} 

